In [3]:
from pandas_datareader import data, wb
import pandas as pd
import numpy as np
import datetime
import csv
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

def predict_clossing_price(start_year, start_month, start_day, code):
    
    start = datetime.datetime(start_year,start_month,start_day)
    end = datetime.date.today()
    stockData = data.DataReader(code , 'yahoo',start,end)

    def get_rsi(sdata,m,mem):
        neg = 0
        pos = 0
        RS = 0
        RSI = 100

        upcloses = 0
        downcloses = 0

        n = m
        k = m-1

        for p in range(mem):
           diff = sdata[n,3] - sdata[k,3]
           if (diff>=0):
            upcloses = upcloses + diff
            pos = pos+1
           else:
            downcloses = downcloses + diff
            neg = neg+1

           n = n-1
           k = k-1

        downcloses = -downcloses
        if(neg == 0):
            return 100
        else:
            RS = (upcloses*neg)/(downcloses*pos)


        RSI = 100 - (100/(1+RS))

        return RSI


    def get_mfi(sdata,m,mem):
        neg = 0
        pos = 0
        MFR = 0
        MFI = 100

        pmflow = 0
        nmflow = 0

        n = m
        k = m-1

        for p in range(mem):

           typ_pricec = (sdata[n,0] + sdata[n,1] + sdata[n,3])/3
           typ_pricep = (sdata[k,0] + sdata[k,1] + sdata[k,3])/3

          # print(typ_price,sdata[n,0],sdata[n,1],sdata[n,3])

           if (typ_pricec>=typ_pricep):
             pmflow = pmflow + ((sdata[n,4])*(typ_pricec))
             pos = pos+1
           else:
             nmflow = nmflow + ((sdata[n,4])*(typ_pricec))
             neg = neg+1

           n = n-1
           k = k-1

        if(neg == 0):
            return 100
        else:
            MFR = pmflow/nmflow

        MFI = 100 - (100/(1+MFR))

        return MFI

    def get_ema(sdata,m,mem,EMAp):

       EMA = sdata[m,3]*(2/(1+mem)) + (1-(2/(1+mem)))*EMAp 
       #print("EMA",EMA)

       return EMA

    def get_so(sdata,m,mem):

       SO = ((sdata[m,3]-sdata[m,1])/(sdata[m,0]-sdata[m,1]))*100
       #print("SO",SO)
       return SO


    memory = 14
    sdat2 = stockData.reset_index()
    del sdat2["Date"]
    del sdat2["Adj Close"]
    sdat3 = np.array(sdat2,dtype=np.float32)

    #print(sdat3[0])

    df1 = pd.DataFrame(columns=['Open','High','Low','Close','Volume','RSI','MFI','EMA','SO','CloseNext'])

    df2 = pd.DataFrame(columns=['Close','RSI','MFI','EMA','SO','CloseNext'])

    arr = np.array(df1.values)

    #print("Printing j\n")

    EMAp = 0
    acc = 0

    for i in range(memory):
        acc = acc + sdat3[i,3]

    EMAp = acc / memory    

    #for i in range(len(sdat3) -memory):
    for i in range(len(sdat3)-1 -memory):
        j = i + memory

        RSI = get_rsi(sdat3,j,memory)
        #print("RSI:",RSI)

        MFI = get_mfi(sdat3,j,memory)
        #print("MFI:",MFI)

        EMA = get_ema(sdat3,j,memory,EMAp)
        EMAp = EMA

        SO = get_so(sdat3,j,memory)

        N_close = sdat3[j+1,3]

        rec1 = [sdat3[j,2],sdat3[j,0],sdat3[j,1],sdat3[j,3],sdat3[j,4],RSI,MFI,EMA,SO,N_close]
        rec2 = [sdat3[j,3],RSI,MFI,EMA,SO,N_close]

        if(sdat3[j,4]!=0):
            d1 = {"Open":sdat3[j,2],"High":sdat3[j,0],"Low":sdat3[j,1],"Close":sdat3[j,3],"Volume":sdat3[j,4],"RSI":RSI,"MFI":MFI,"EMA":EMA,"SO":SO,"CloseNext":N_close}
            df1.loc[i] = rec1
            df2.loc[i] = rec2

    #ANN
    dataset = df1
    y = pd.DataFrame(dataset['CloseNext'])
    X = dataset.drop(['CloseNext'], axis = 1)

    X = np.array(X)
    y = np.array(y)
    #X = X[1700:2030,:]
    #y = y[1700:2030,:]
    y = y.flatten()

    #Feature scaling
    scaled = StandardScaler()
    scaled.fit(X)
    X = scaled.transform(X)

    #Train Test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

    ## ANN implementation
    # define base model of our neural network for regression taks
    def endgame():
        # Adding the neurons in various layers
        model = Sequential()
        model.add(Dense(9, input_dim=9, kernel_initializer='normal', activation='relu'))
        model.add(Dense(15, kernel_initializer='normal', activation='relu'))
        model.add(Dense(8, kernel_initializer='normal', activation='relu'))
        model.add(Dense(1, kernel_initializer='normal'))
        # Compile model for our use in KerasRegressor
        model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mape'])
        return model

    ann_regression = KerasRegressor(build_fn = endgame, epochs=100, batch_size=5, verbose=1)

    ann_regression.fit(X_train,y_train)

    ann_predict = ann_regression.predict(X_test)

    error = mean_absolute_error(ann_predict,y_test)
    per_err = (error/np.mean(y_test)) * 100
    print('The mean absolute error is {} and percentage error is {}.'.format(error,per_err))
    
    return ann_regression.predict(scaled.transform([df1.iloc[-1,:-1].values.tolist()])), df1.iloc[-1,:], error,per_err

In [4]:
#parameter
start_year = 2011
start_month = 1
start_day = 1
code = "IRFC.NS"

#predicted_price, df1  = predict_clossing_price(start_year, start_month, start_day, code)

In [5]:
list_of_stock_code = ["IOC.NS", "INFY.NS", "HCLTECH.NS", "SBIN.NS", "SBICARD.NS", "SBILIFE.NS", "ITC.NS", "TECHM.NS", 
                     "RELIANCEPP-E1.NS", "WIPRO.NS", "RELIANCE.NS", "IBM", "GOOG", "HINDUNILVR.NS", "HINDPETRO.NS",
                      "HINDALCO.NS", "HDFCAMC.NS", "HDFC.NS", "BANKBARODA.NS", "YESBANK.NS", "ASHOKLEY.NS", "MOTHERSUMI.NS",
                      "IRCTC.NS", "BEPL.NS", "TATAPOWER.NS", "BAJFINANCE.NS", "ONGC.NS", "SUNPHARMA.NS", "COALINDIA.NS", 
                      "ADANIGREEN.NS", "DRREDDY.NS", "JUBLFOOD.NS", "WESTLIFE.NS", "TCS.NS", "TATACONSUM.NS", "SRTRANSFIN.NS"]

list_of_stock_code = np.unique(list_of_stock_code)

#parameter
start_year = 2011
start_month = 1
start_day = 1

data_values = []

error_code = []
for code in list_of_stock_code:
    try:
        predicted_price, df1, error,per_err  = predict_clossing_price(start_year, start_month, start_day, code)
        data_values.append([
            code, 
            round(float(df1.Close),2),
            round(float(predicted_price),2),
            round(float(error),2),
            round(float(per_err),2),
            0])
    except:
        error_code.append(code)

<ipython-input-3-f7748bae37fd>:104: RuntimeWarning: invalid value encountered in float_scalars
  SO = ((sdata[m,3]-sdata[m,1])/(sdata[m,0]-sdata[m,1]))*100


Epoch 1/100
92/92 [==============================] - 1s 2ms/step - loss: nan - mape: nan     
Epoch 2/100
92/92 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 3/100
92/92 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 4/100
92/92 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 5/100
92/92 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 6/100
92/92 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 7/100
92/92 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 8/100
92/92 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 9/100
92/92 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 10/100
92/92 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 11/100
92/92 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 12/100
9

92/92 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 93/100
92/92 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 94/100
92/92 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 95/100
92/92 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 96/100
92/92 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 97/100
92/92 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 98/100
92/92 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 99/100
92/92 [==============================] - 0s 2ms/step - loss: nan - mape: nan
Epoch 100/100
40/40 [==============================] - 0s 1ms/step


<ipython-input-3-f7748bae37fd>:104: RuntimeWarning: invalid value encountered in float_scalars
  SO = ((sdata[m,3]-sdata[m,1])/(sdata[m,0]-sdata[m,1]))*100


Epoch 1/100
349/349 [==============================] - 1s 2ms/step - loss: 5428.1928 - mape: 91.1591
Epoch 2/100
349/349 [==============================] - 1s 2ms/step - loss: 317.3756 - mape: 26.8799
Epoch 3/100
349/349 [==============================] - 1s 2ms/step - loss: 122.8655 - mape: 18.9069
Epoch 4/100
349/349 [==============================] - 1s 2ms/step - loss: 43.8869 - mape: 13.6682
Epoch 5/100
349/349 [==============================] - 1s 2ms/step - loss: 18.2209 - mape: 9.5137
Epoch 6/100
349/349 [==============================] - 1s 2ms/step - loss: 7.2504 - mape: 4.9831
Epoch 7/100
349/349 [==============================] - 1s 1ms/step - loss: 5.0931 - mape: 3.9334
Epoch 8/100
349/349 [==============================] - 1s 2ms/step - loss: 4.5421 - mape: 3.5694: 0s - loss: 4.6812 - 
Epoch 9/100
349/349 [==============================] - 0s 1ms/step - loss: 4.9667 - mape: 2.9502
Epoch 10/100
349/349 [==============================] - 1s 1ms/step - loss: 4.4315 - mape: 2

...

Epoch 1/100
349/349 [==============================] - 1s 1ms/step - loss: 43490.7463 - mape: 97.5288
Epoch 2/100
349/349 [==============================] - 0s 1ms/step - loss: 8952.1322 - mape: 40.9237
Epoch 3/100
349/349 [==============================] - 0s 1ms/step - loss: 2588.0159 - mape: 20.8665
Epoch 4/100
349/349 [==============================] - 0s 1ms/step - loss: 546.5359 - mape: 8.0918
Epoch 5/100
349/349 [==============================] - 0s 1ms/step - loss: 170.3673 - mape: 4.6767
Epoch 6/100
349/349 [==============================] - 0s 1ms/step - loss: 63.7624 - mape: 2.8583
Epoch 7/100
349/349 [==============================] - 0s 1ms/step - loss: 28.1023 - mape: 1.9348
Epoch 8/100
349/349 [==============================] - 0s 1ms/step - loss: 22.0603 - mape: 1.6654
Epoch 9/100
349/349 [==============================] - 0s 1ms/step - loss: 18.0031 - mape: 1.5445
Epoch 10/100
349/349 [==============================] - 0s 1ms/step - loss: 15.6898 - mape: 1.4550
Epoch 1

349/349 [==============================] - 0s 1ms/step - loss: 14.8031 - mape: 1.3105
Epoch 84/100
349/349 [==============================] - 0s 1ms/step - loss: 14.4930 - mape: 1.2250
Epoch 85/100
349/349 [==============================] - 0s 1ms/step - loss: 12.4559 - mape: 1.2070
Epoch 86/100
349/349 [==============================] - 0s 1ms/step - loss: 14.4338 - mape: 1.2428
Epoch 87/100
349/349 [==============================] - 0s 1ms/step - loss: 13.6271 - mape: 1.2610
Epoch 88/100
349/349 [==============================] - 0s 1ms/step - loss: 16.3780 - mape: 1.3973
Epoch 89/100
349/349 [==============================] - 0s 1ms/step - loss: 13.0935 - mape: 1.2320
Epoch 90/100
349/349 [==============================] - 0s 1ms/step - loss: 15.8456 - mape: 1.3309
Epoch 91/100
349/349 [==============================] - 0s 1ms/step - loss: 14.5063 - mape: 1.2664
Epoch 92/100
349/349 [==============================] - 0s 1ms/step - loss: 12.9859 - mape: 1.2391
Epoch 93/100
349/349 [=

<ipython-input-3-f7748bae37fd>:104: RuntimeWarning: invalid value encountered in float_scalars
  SO = ((sdata[m,3]-sdata[m,1])/(sdata[m,0]-sdata[m,1]))*100


Epoch 1/100
349/349 [==============================] - 1s 1ms/step - loss: nan - mape: nan
Epoch 2/100
349/349 [==============================] - 1s 1ms/step - loss: nan - mape: nan
Epoch 3/100
349/349 [==============================] - 1s 1ms/step - loss: nan - mape: nan
Epoch 4/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 5/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 6/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 7/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 8/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 9/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 10/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 11/100
349/349 [==============================] - 1s 1ms/step - loss: nan - mape: n

349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 91/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 92/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 93/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 94/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 95/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 96/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 97/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 98/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 99/100
349/349 [==============================] - 0s 1ms/step - loss: nan - mape: nan
Epoch 100/100
150/150 [==============================] - 0s 1ms/step


In [6]:
dataset = pd.DataFrame(columns = ["symbol", "Clossing_Price", "Predicted_Price","Error", "Error_in_per", "Actual_Price"], data = data_values)
dataset.to_csv("stock_price_prediction_13_march_2021.csv")

In [7]:
dataset

,symbol,Clossing_Price,Predicted_Price,Error,Error_in_per,Actual_Price
0,ASHOKLEY.NS,123.90,124.41,1.30,1.91,0
1,BAJFINANCE.NS,5545.05,5453.51,25.88,1.99,0
2,BANKBARODA.NS,80.00,77.95,2.73,1.99,0
3,COALINDIA.NS,150.95,148.45,4.42,1.52,0
4,DRREDDY.NS,4500.40,4536.87,36.68,1.36,0
5,GOOG,2114.77,2112.03,9.77,1.24,0
6,HCLTECH.NS,988.50,986.57,5.47,1.38,0
7,HDFC.NS,2599.80,2617.71,19.30,1.39,0
8,HDFCAMC.NS,3118.50,3094.21,43.26,1.86,0
9,HINDALCO.NS,340.35,337.59,3.01,1.88,0


In [ ]:
for code in list_of_stock_code[:2]:
    print(code)

In [ ]:
start = datetime.datetime(start_year,start_month,start_day)
end = datetime.date.today()
stockData = data.DataReader(list_of_stock_code[0] , 'yahoo',start,end)

In [ ]:
stockData